## Projeto 2 - Ciência dos dados

## Integrantes: 

Luca Guazzelli 

Pedro Henrique Vidal 

Rafael Agnelo

## Objetivo: 

A base de dados é uma base de microdados do ENEM em relação a dados do ENEM por escolas (públicas e privadas) no Brasil. Em relação à isso, faz-se a seguinte pergunta temática:

A partir de uma série de features (Localidade, Taxa de abandono, Porte da escola , Média das notas em matemática, Tipo da dependência administrativa, Número de participantes do enem que cumprem os requisitos de participação e INSE), qual é o perfil médio de uma escola que performa mal, isto é, de forma insatisfatória em Matemática no ENEM (Média inferior à 500) no ano de 2015?



## Importando Bibliotecas

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## Carregando base de dados

In [9]:
dados = pd.read_csv('microdados_enem_por_escola\DADOS\MICRODADOS_ENEM_ESCOLA.csv',encoding='latin1', sep=";")

C:\Users\lucah\AppData\Local\Temp\ipykernel_96288\2310550405.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv('microdados_enem_por_escola\DADOS\MICRODADOS_ENEM_ESCOLA.csv',encoding='latin1', sep=";")


In [29]:
dados.head(5)

,NU_ANO,CO_UF_ESCOLA,SG_UF_ESCOLA,CO_MUNICIPIO_ESCOLA,NO_MUNICIPIO_ESCOLA,CO_ESCOLA_EDUCACENSO,NO_ESCOLA_EDUCACENSO,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,...,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,INSE,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO,PORTE_ESCOLA
0,2007,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,144,...,NaN,NaN,69.03,NaN,NaN,NaN,91.9,8.1,0.0,Maior que 90 alunos
1,2006,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,184,...,NaN,NaN,57.82,NaN,NaN,NaN,NaN,NaN,NaN,Maior que 90 alunos
2,2005,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,220,...,NaN,NaN,64.83,NaN,NaN,NaN,86.5,12.4,1.1,Maior que 90 alunos
3,2008,11,RO,1100205,Porto Velho,11000058,CENTRO DE ENSINO CLASSE A,4,1,186,...,72.16,60.02,NaN,NaN,NaN,NaN,90.3,9.7,0.0,Maior que 90 alunos
4,2007,11,RO,1100205,Porto Velho,11000171,CENTRO EDUCACIONAL MOJUCA,4,1,19,...,NaN,NaN,58.84,NaN,NaN,NaN,74.2,21.0,4.8,De 1 a 30 alunos


## Filtrando a base de dados em base de fetures 

In [30]:
dados_limpando = dados.loc[dados["NU_ANO"] == 2015,("SG_UF_ESCOLA", "TP_LOCALIZACAO_ESCOLA", "NU_TAXA_ABANDONO", "PORTE_ESCOLA", "NU_MEDIA_MT", "INSE")]

dados_limpando.head(5)

,SG_UF_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_TAXA_ABANDONO,PORTE_ESCOLA,NU_MEDIA_MT,INSE
67619,RO,1,0.0,Maior que 90 alunos,627.66,Grupo 6
67628,RO,1,0.0,De 1 a 30 alunos,459.72,Grupo 4
67640,RO,1,0.0,De 31 a 60 alunos,507.22,Grupo 5
67642,RO,1,0.8,De 31 a 60 alunos,529.87,Grupo 5
67648,RO,1,2.4,De 1 a 30 alunos,528.93,Grupo 5


## Analise Exploratória